In [2]:
from orbit_generation_testing.reader import *
from orbit_generation_testing.processing import *

In [ ]:
%load_ext autoreload
%autoreload 2


import numpy as np
import sklearn
import pandas as pd
import copy
import sklearn.model_selection
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import tsgm

# Get Data

In [ ]:
from pathlib import Path
orbit_directory = Path(r"C:\Users\alvar\Desktop\Orbit\Data")
orbit_data, labels = get_orbit_data_and_labels(data_directory=orbit_directory)

In [6]:
data_path = r"C:/Users/alvar/Documents/GitHub/orbit-generation-testing/data/orbit_data_by_period.h5"
data = load_orbit_data(data_path, dataset_path='orbit_data')

# Train VAE

In [7]:
%load_ext autoreload
%autoreload 2

import numpy as np
import sklearn
import copy
import sklearn.model_selection
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import tsgm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
hdf5_file_path = r"C:\Users\alvar\Desktop\Orbit\Data\em_orbits.h5"
dataset_path = '/files/PERIODIC ORBITS'
orbit_data = load_orbit_data(hdf5_file_path, dataset_path=dataset_path)
reshaped_array = orbit_data.reshape(36071, 7, 5, 1500)
orbit_data = reshaped_array.transpose(0, 2, 1, 3).reshape(36071 * 5, 7, 1500)
np.save('splitted_orbits.npy', orbit_data)
orbit_data.shape

(36071, 7, 7500)

In [3]:
orbit_data = np.load('splitted_orbits.npy')

Params

In [4]:
latent_dim = 32
feature_dim = 7
seq_len = 500
batch_size = 32
epochs = 10

In [5]:
resampled_orbits = np.transpose(resampled_orbits, (0, 2, 1))

In [6]:
architecture = tsgm.models.zoo["vae_conv5"](seq_len=seq_len, feat_dim=feature_dim, latent_dim=latent_dim)
encoder, decoder = architecture.encoder, architecture.decoder

In [7]:
scaler = tsgm.utils.TSFeatureWiseScaler()        
scaled_data = scaler.fit_transform(resampled_orbits) ####
scaled_data.shape

(180355, 500, 7)

In [8]:
vae = tsgm.models.cvae.BetaVAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())

vae.fit(scaled_data, epochs=epochs, batch_size=batch_size)

Epoch 1/10
   2/5637 [..............................] - ETA: 313:39:11 - loss: 2834.4039 - reconstruction_loss: 2713.4011 - kl_loss: 0.5689

In [ ]:
x_decoded = vae.predict(scaled_data)

In [ ]:
generation = vae.generate(100)
generation.shape

In [ ]:
tsgm.utils.visualize_original_and_reconst_ts(scaled_data, x_decoded, num=10)

In [ ]:
# Encode data into the latent space
latent_representations = encoder.predict(scaled_data)

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# Encode data into the latent space
latent_mean, _, _ = vae.encoder(scaled_data)
latent_representations = latent_mean.numpy()  # Convert tensor to numpy array

# Use t-SNE to reduce dimensionality for visualization
tsne = TSNE(n_components=2, random_state=42)
tsne_results = tsne.fit_transform(latent_representations)

# Plot the t-SNE output
plt.figure(figsize=(10, 7))
plt.scatter(tsne_results[:, 0], tsne_results[:, 1], alpha=0.5)
plt.xlabel('t-SNE dimension 1')
plt.ylabel('t-SNE dimension 2')
plt.title('Latent Space Visualization')
plt.show()